In [2]:
import pickle
import csv

In [3]:
from collections import defaultdict

In [4]:
GRAPH_PICKLE_PATH = "./graph.pickle"
GRAPH_CSV_PATH = "./graph.csv"
GRAPH_BIG_INCOME_NODES_CSV_PATH = "./graph_big_income_nodes.csv"
GRAPH_TOP_INCOME_NODES_CSV_PATH = "./graph_top_income_nodes.csv"
GRAPH_TOP_OUTCOME_NODES_CSV_PATH = "./graph_top_outcome_nodes.csv"
GRAPH_BIGGEST_COMPONENT_CSV_PATH = "./graph_biggest_component.csv"

In [45]:
def load_graph():
    with open(GRAPH_PICKLE_PATH, 'rb') as graph_file:
        graph = []
        nodes = {}
        
        def get_node(link, index):
            if link not in nodes:
                nodes[link] = index
                index += 1
            return nodes[link], index
        
        index = 0
        for link_from, links_to in pickle.load(graph_file).items():
            for link_to in links_to:
                node_from, index = get_node(link_from, index)
                node_to, index = get_node(link_to, index)
                graph.append((node_from, node_to))
        return graph

In [6]:
def build_graph_in_csv(output_path, graph):
    with open(output_path, 'w') as csvfile:
        graph_writer = csv.writer(csvfile, delimiter=';')
        for link_from, link_to in graph:
                graph_writer.writerow([link_from, link_to])

In [7]:
def leave_only_big_income_degree_nodes(graph, income_degree_threshold=200):
    result_graph = []
    freqs = defaultdict(lambda: 0)
    for link_from, link_to in graph:
        freqs[link_to] += 1
    for link_from, link_to in graph:
        if (freqs[link_from] >= income_degree_threshold) and (freqs[link_to] >= income_degree_threshold):
            result_graph.append((link_from, link_to))
    return result_graph

In [8]:
def leave_only_top_nodes_by_income_degree(graph, top_amount=200):
    result_graph = []
    freqs = defaultdict(lambda: 0)
    for link_from, link_to in graph:
        freqs[link_to] += 1

    all_freqs = freqs.values()
    threshold = sorted(all_freqs)[-top_amount]
    for link_from, link_to in graph:
        if (freqs[link_from] >= threshold) and (freqs[link_to] >= threshold):
            result_graph.append((link_from, link_to))
    return result_graph

In [9]:
def leave_only_top_nodes_by_outcome_degree(graph, top_amount=200):
    result_graph = []
    freqs = defaultdict(lambda: 0)
    for link_from, link_to in graph:
        freqs[link_from] += 1

    all_freqs = freqs.values()
    threshold = sorted(all_freqs)[-top_amount]
    for link_from, link_to in graph:
        if (freqs[link_from] >= threshold) and (freqs[link_to] >= threshold):
            result_graph.append((link_from, link_to))
    return result_graph

In [10]:
def dfs(current, edges, used, order):
    used[current] = True
    for next_node in edges[current]:
        if not used[next_node]:
            dfs(next_node, edges, used, order)
    order.append(current)
    
def dfs_reversed(current, edges, used, components):
    used[current] = True
    components[-1].append(current)
    for next_node in edges[current]:
        if not used[next_node]:
            dfs_reversed(next_node, edges, used, components)
            

def get_components_of_strong_connectivity(graph):
    edges = defaultdict(lambda: [])
    rev_edges = defaultdict(lambda: [])
    for link_from, link_to in graph:
        edges[link_from].append(link_to)
        rev_edges[link_to].append(link_from)
    used = defaultdict(lambda: False)
    order = []
    all_nodes = set(edges.keys())
    for nodes in edges.values():
        all_nodes.update(nodes)
    for link_from in all_nodes:
        if not used[link_from]:
            dfs(link_from, edges, used, order)
    used = defaultdict(lambda: False)
    components = []
    for link in reversed(order):
        if not used[link]:
            components.append([])
            dfs_reversed(link, rev_edges, used, components)
    return components

In [19]:
def leave_only_biggest_strong_connectivity_component(graph):
    components = get_components_of_strong_connectivity(graph)
    max_size = max(map(len, components))
    max_component = []
    for component in components:
        if len(component) == max_size:
            max_component = set(component)
            break
    result_graph = []
    for link_from, link_to in graph:
        if link_from in max_component:  # link_to is in it too
            result_graph.append((link_from, link_to))
    return result_graph

In [36]:
build_graph_in_csv(GRAPH_CSV_PATH, load_graph())

In [30]:
build_graph_in_csv(GRAPH_BIG_INCOME_NODES_CSV_PATH, leave_only_big_income_degree_nodes(load_graph()))

In [43]:
build_graph_in_csv(GRAPH_TOP_INCOME_NODES_CSV_PATH, leave_only_top_nodes_by_income_degree(load_graph()))

In [44]:
build_graph_in_csv(GRAPH_TOP_OUTCOME_NODES_CSV_PATH, leave_only_top_nodes_by_outcome_degree(load_graph()))

In [38]:
build_graph_in_csv(GRAPH_BIGGEST_COMPONENT_CSV_PATH, leave_only_biggest_strong_connectivity_component(load_graph()))